In [ ]:
"""
Created by Leo Dettori on 2021.07.22.
"""

def find_parameters(experiment, output):
    
    import numpy as np
    import matplotlib.pyplot as plt
    from scipy import interpolate
    import math

    #Starts to loop through the wavelengths
    for i1 in experiment:

        #starts to loop through the samples for each wavelength
        for i2 in experiment[str(i1)]:  

                #starts to loop through the temeperature programs for each sample for each wavelength
                for i3 in experiment[str(i1)][str(i2)]:
                    
                    
                    #Interpolating with cubic spline and find derivatives

                    #Defining x and y   #if we're doing cooling, we need to re-arrange data from lowest to highest temp
                    if experiment[str(i1)][str(i2)][str(i3)] == 'Cooling':
                        x = list(reversed(experiment['320.00']['80uL_RBM3_C_IDR+20uL_Water']['Cooling']["Temp"]))
                        y = list(reversed(experiment['320.00']['80uL_RBM3_C_IDR+20uL_Water']['Cooling']["Abs"]))

                    #Defining x and y when heating
                    if experiment[str(i1)][str(i2)][str(i3)] == 'Heating':
                        x = experiment['320.00']['80uL_RBM3_C_IDR+20uL_Water']['Heating']["Temp"]
                        y = experiment['320.00']['80uL_RBM3_C_IDR+20uL_Water']['Heating']["Abs"]

                    #print("x = " + str(x))
                    #print("y = " + str(y))

                    #Finding recommend smoothness for the cubic spline interpolation
                    m = len(y)
                    rec_s=m-math.sqrt(2*m)

                    #Interpolating with the cubic spline
                    tck = interpolate.splrep(x, y, s= rec_s)  #Initially it was s = 0
                    #Updates x and y with the interpolation results
                    xnew = np.arange(x[0], x[-1], abs(x[0]-x[1])/100)   #np.arrange creates an array eg.: np.arange([start, ]stop, [step, ], dtype=None)
                    ynew = interpolate.splev(xnew, tck, der=0)


                    print("\n")


                    #Plotting results
                    fig, axs = plt.subplots(2, 2, figsize =(30, 23))
                    axs[0, 0].plot(x, y, 'o' , markersize=10)
                    axs[0, 0].plot(xnew, ynew, 'b', linewidth=3)
                    axs[0, 0].grid('on')
                    axs[0, 0].legend(['Data', 'Cubic Spline'], fontsize=22)
                    axs[0, 0].set_xlabel('Temperature', fontsize=24)
                    axs[0, 0].set_ylabel('Scattering', fontsize=24)
                    axs[0, 0].set_title('Cubic Spline Interpolation', fontsize=35)


                    print("\n")


                    #First Derivative of Interpoalted (Spline) data
                    yder = interpolate.splev(xnew, tck, der=1)

                    axs[1, 0].plot(xnew, yder, color='orange', linewidth=3)
                    axs[1, 0].grid('on')
                    axs[1, 0].legend(['1st Derivative'], fontsize=22)
                    axs[1, 0].set_xlabel('Temperature', fontsize=24)
                    axs[1, 0].axhline(y=0, color='r', linestyle='--', linewidth=2.5)
                    axs[1, 0].set_title('1st Derivative Estimation', fontsize=35)



                    #Reporting the Critical points:
                    #For this, we interpolate the first derivative of the data and look for its roots
                    #Prepare the smoothness coefficient for the new cubic spline interpolation
                    m_1 = len(yder)
                    rec_s_1 = m_1-math.sqrt(2*m_1)
                    #Finds the Critical points
                    critical_points = interpolate.sproot(interpolate.splrep(xnew, yder, s= rec_s_1))
                    #Reports the Cloud Point(s)
                    print("The " + str(len(critical_points)) +" Critical points are: " + str(critical_points).strip('[]'))
                    #print(yder)



                    print("\n")

                    #Second Derivative of Interpolated (Spline) data
                    yder_2 = interpolate.splev(xnew, tck, der=2)
                    axs[1, 1].plot(xnew, yder_2, color='green', linewidth=3)
                    axs[1, 1].grid('on')
                    axs[1, 1].legend(['2nd Derivative'], fontsize=22)
                    axs[1, 1].set_xlabel('Temperature', fontsize=24)
                    axs[1, 1].axhline(y=0, color='r', linestyle='--', linewidth=2.5)
                    axs[1, 1].set_title('2nd Derivative Estimation', fontsize=35)


                    #Reporting the Inflection/Cloud point:
                    #For this, we interpolate the second derivative of the data and look for its roots
                    #Prepare the smoothness coefficient for the new cubic spline interpolation
                    m_2 = len(yder_2)
                    rec_s_2 = m_2-math.sqrt(2*m_2)
                    #Finds the Cloud Point
                    cloud_point = interpolate.sproot(interpolate.splrep(xnew, yder_2, s= rec_s_2))
                    #Reports the Cloud Point(s)
                    print("Temperature at the Inflection/Cloud point is: " + str(cloud_point).strip('[]'))


                    print("\n")


                    #Give temperature and get absorbance (now for cloud_point)
                    #Choose your x:
                    x_give = cloud_point
                    y_given = interpolate.splev(x_give, tck, der=0, ext=0)   # if ext=0, return the extrapolated value.

                    print("For the Cloud Point Temperature (" + str(x_give).strip('[]') + "), the Scattering is " + str(y_given).strip('[]'))

                    print("\n")


                    #Calculating delta Temperature (Tfinal - Tinitial) and delta Scattering (Absfinal - Absinitial)
                    if critical_points[0] != '' and critical_points[-1] != '':                         
                        delta_T = critical_points[-1] - critical_points[0]
                        delta_Abs = interpolate.splev(critical_points[-1], tck, der=0, ext=0) - interpolate.splev(critical_points[0], tck, der=0, ext=0)

                        print("Delta Temperature is "+str(delta_T))
                        print("Delta Scattering is "+str(delta_Abs))
                        print("\n")
                        

                    #Calculating LLPS_Capacity
                    if critical_points[0] != '' and critical_points[-1] != '': 
                        LLPS_Cap = interpolate.splint(critical_points[0], critical_points[-1], tck)

                        print("LLPS Capacity is: " +str(LLPS_Cap))
                        print("\n")


                    #Plotting Summary Plot
                    axs[0, 1].plot(xnew, ynew, 'b', linewidth=3)
                    axs[0, 1].grid('on')


                    #if cloud_point != '':
                    if cloud_point[0] != '':
                        axs[0, 1].plot(x_give, y_given, '.', color='green', markersize=35)
                        #plt.legend(['Cloud Point'], fontsize=12)

                    #Plotting lines on initial and final temperatures

                    if critical_points[0] != '':
                        axs[0, 1].axvline(x=critical_points[0], color='orange', linestyle='--', linewidth=4)

                    if critical_points[-1] != '':
                        axs[0, 1].axvline(x=critical_points[-1], color='orange', linestyle='--', linewidth=4)


                    axs[0, 1].set_xlabel('Temperature', fontsize=24)
                    axs[0, 1].set_ylabel('Scattering', fontsize=24)
                    axs[0, 1].set_title('Summary ', fontsize=38)


                    axs[0, 1].legend(['Cubic Spline','Cloud Point', 'Transtion Temperatures'], fontsize=22)
                    plt.rc('xtick', labelsize=18) 
                    plt.rc('ytick', labelsize=18) 
                    fig.savefig(output + "/" + str(experiment[str(i1)][str(i2)][str(i3)]["Name"]) +".png")

